Rappel du format des données : <br>
mains_2J = [preflop, flop, turn, river, proba, small, bigblind,BB, my_stack, stack_J2, my_bet, bet_J2, pot, decision, decision_value]

## Récupération des mains à partir des fichiers

In [1]:
import numpy as np
import torch

import torch.utils.data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


BATCH_SIZE = 1024

In [2]:
mains_2J = np.load('mains2J.npy')

In [3]:
#creation d'un mask pour detecter les all in ( > 4)
list_allin_2J = np.where(mains_2J[:,14] > 4)[0]

# On supprimer les all-ins : ils seront gérés par un tableau de psuh or fold
mains_2J = np.delete(mains_2J, list_allin_2J, 0)

## Séparation des données

In [4]:
nb_features = mains_2J.shape[1]
mains_2J_X = mains_2J[:,:(nb_features-2)]
mains_2J_Y = mains_2J[:, (nb_features-2):(nb_features-1)].squeeze()

scaler = StandardScaler()
scaler.fit(mains_2J_X[:,[4,7,8,9,10,11,12]])

mains_2J_X[:,[4,7,8,9,10,11,12]] = scaler.transform(mains_2J_X[:,[4,7,8,9,10,11,12]])

mains_2J_X_train, mains_2J_X_val, mains_2J_Y_train, mains_2J_Y_val = train_test_split(mains_2J_X, mains_2J_Y,
                                                                     test_size=0.01, stratify=mains_2J_Y)

In [5]:
np.savez('mean_var_2J.npz', mean=scaler.mean_, var=scaler.var_)

## Datasets et dataloaders

In [127]:
class MainsDataset():
    def __init__(self, X, Y):
        self.hands= torch.from_numpy(X).type(torch.cuda.DoubleTensor)
        self.labels= torch.from_numpy(Y).type(torch.cuda.LongTensor)
        
    def __getitem__(self, index):
        hand =  self.hands[index]
        label =  self.labels[index]
        return (hand, label)

    def __len__(self):
        count = len(self.hands)
        return count 

In [128]:
train_mains_2J = MainsDataset(mains_2J_X_train, mains_2J_Y_train)
train_mains_2J_loader = torch.utils.data.DataLoader(train_mains_2J, batch_size=BATCH_SIZE, shuffle=True)

val_mains_2J = MainsDataset(mains_2J_X_val, mains_2J_Y_val)
val_mains_2J_loader = torch.utils.data.DataLoader(val_mains_2J, batch_size=BATCH_SIZE, shuffle=True)


## Construction du réseau de neurones

In [134]:
class NN_2J(nn.Module):

    def __init__(self):
        super(NN_2J, self).__init__()
        self.largeur = 150
        self.input = nn.Linear(13, self.largeur)
        self.hidden = nn.Linear(self.largeur, self.largeur)
        self.final = nn.Linear(self.largeur, 3)


    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = self.final(x)
        return x

NN_2J = NN_2J()
print(NN_2J)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NN_2J.to(device).type(torch.cuda.DoubleTensor)

NN_2J(
  (input): Linear(in_features=13, out_features=150, bias=True)
  (hidden): Linear(in_features=150, out_features=150, bias=True)
  (final): Linear(in_features=150, out_features=3, bias=True)
)


NN_2J(
  (input): Linear(in_features=13, out_features=150, bias=True)
  (hidden): Linear(in_features=150, out_features=150, bias=True)
  (final): Linear(in_features=150, out_features=3, bias=True)
)

## Entrainement du réseau 

In [147]:

criterion = nn.CrossEntropyLoss()

lr_tab = [0.1, 0.08, 0.06, 0.04, 0.02, 0.01]
# lr_tab = [0.001]

for lr_ in lr_tab:
    
    optimizer = optim.SGD(NN_2J.parameters(), lr=lr_)
    for epoch in range(30):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_mains_2J_loader, 0):
            # get the inputs
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = NN_2J(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('lr = %.3f  |  %d loss: %.3f' %(lr_ ,epoch + 1,running_loss*BATCH_SIZE))

print('Finished Training')

lr = 0.001  |  1 loss: 26863.539
lr = 0.001  |  2 loss: 26769.946
lr = 0.001  |  3 loss: 26758.515
lr = 0.001  |  4 loss: 26758.988
lr = 0.001  |  5 loss: 26745.926
lr = 0.001  |  6 loss: 26771.577
lr = 0.001  |  7 loss: 26771.821
lr = 0.001  |  8 loss: 26758.763
lr = 0.001  |  9 loss: 26749.989
lr = 0.001  |  10 loss: 26758.299
lr = 0.001  |  11 loss: 26756.427
lr = 0.001  |  12 loss: 26745.518
lr = 0.001  |  13 loss: 26753.161
lr = 0.001  |  14 loss: 26765.019
lr = 0.001  |  15 loss: 26737.696
lr = 0.001  |  16 loss: 26743.278
lr = 0.001  |  17 loss: 26768.237
lr = 0.001  |  18 loss: 26738.709
lr = 0.001  |  19 loss: 26764.233
lr = 0.001  |  20 loss: 26749.925
lr = 0.001  |  21 loss: 26732.131
lr = 0.001  |  22 loss: 26757.474
lr = 0.001  |  23 loss: 26729.024
lr = 0.001  |  24 loss: 26746.797
lr = 0.001  |  25 loss: 26737.329
lr = 0.001  |  26 loss: 26724.107
lr = 0.001  |  27 loss: 26741.487
lr = 0.001  |  28 loss: 26740.044
lr = 0.001  |  29 loss: 26738.797
lr = 0.001  |  30 loss:

## Test du réseau

In [146]:
correct = 0
total = 0


classes = ['Fold', 'Call', 'Raise']
class_correct = [0, 0, 0]
class_total = [1, 1, 1]


with torch.no_grad():
    for data in val_mains_2J_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = NN_2J(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print('Accuracy of the network : %d %%' % (100 * correct / total))

print('Accuracy for each class :')

for i in range(3):
    print('%s : %d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

    
# 5 hiddens de 150 :   80% |  85/85/63

Accuracy of the network : 74 %
Accuracy for each class :
Fold : 75 %
Call : 81 %
Raise : 53 %


## Sauvegarde du réseau 

In [148]:
model_ft = NN_2J.state_dict()
filename = 'model_theo_2J.pth'
torch.save(model_ft, filename)